In [ ]:
import pandas as pd
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
df_ref = pd.read_excel("trainProjeNew.xlsx")

le_hedef = LabelEncoder()
le_hedef.fit(df_ref['cezaDurumu'])

if 'adliSicil' in df_ref.columns:
    df_ref = df_ref.drop(columns=['adliSicil'])

if 'cezaDurumu' in df_ref.columns:
    X_ref_raw = df_ref.drop(columns=['cezaDurumu'])
else:
    X_ref_raw = df_ref.copy()

In [ ]:
yol = "egitilmis_modeller/"

tfidf_sanik = joblib.load(yol + 'tfidf_sanik.pkl')
model_sanik = joblib.load(yol + 'model_sanik.pkl')
le_sanik = joblib.load(yol + 'le_sanik.pkl')

tfidf_avukat = joblib.load(yol + 'tfidf_avukat.pkl')
model_avukat = joblib.load(yol + 'model_avukat.pkl')
le_avukat = joblib.load(yol + 'le_avukat.pkl')

In [ ]:
style = {'description_width': 'initial'}

widget_listesi = {}

In [ ]:
yasakli_sutunlar = [
    'cezaDurumu',
    'malinDegeriTL',    # Slider
    'failYasi',         # Slider
    'sanikTutumu',      # NLP
    'avukatStratejisi'  # NLP
]

In [ ]:
if not X_ref_raw.empty:
    for col in X_ref_raw.columns:
        if col not in yasakli_sutunlar and X_ref_raw[col].dtype == 'object':
            secenekler = X_ref_raw[col].unique().tolist()
            widget_listesi[col] = widgets.Dropdown(options=secenekler, description=f'{col}:', style=style)

In [ ]:
w_malinDegeri = widgets.IntText(value=5000, description='Malın Değeri (TL):', style=style)
w_failYasi = widgets.IntSlider(value=25, min=12, max=90, description='Fail Yaşı:', style=style)

In [ ]:
w_sanikIfadesi = widgets.Textarea(
    description='Sanık İfadesi:',
    placeholder='Sanık savunmasını buraya giriniz...',
    layout=widgets.Layout(width='95%', height='80px'),
    style=style
)

In [ ]:
w_avukatSavunmasi = widgets.Textarea(
    description='Avukat Savunması:',
    placeholder='Avukat savunmasını buraya giriniz...',
    layout=widgets.Layout(width='95%', height='80px'),
    style=style
)

In [ ]:
btn_tahmin = widgets.Button(description="CEZAYI TAHMİN ET", button_style='danger', icon='gavel')
out_panel = widgets.Output()

In [ ]:
def tahmin_et(b):
    with out_panel:
        clear_output()
        print("🔍 Veriler analiz ediliyor...")

        # --- NLP KISMI ---
        try:
            s_txt = w_sanikIfadesi.value if w_sanikIfadesi.value.strip() else " "
            s_vec = tfidf_sanik.transform([s_txt])
            s_cat = le_sanik.inverse_transform(model_sanik.predict(s_vec))[0]

            a_txt = w_avukatSavunmasi.value if w_avukatSavunmasi.value.strip() else " "
            a_vec = tfidf_avukat.transform([a_txt])
            a_cat = le_avukat.inverse_transform(model_avukat.predict(a_vec))[0]

            print(f" NLP Analizi -> Sanık: {s_cat} | Avukat: {a_cat}")
        except:
            s_cat, a_cat = "Pisman", "DelilYetersizligi"
            print(" NLP Hatası: Varsayılan değerler kullanıldı.")

        # --- VERİ BİRLEŞTİRME ---
        yeni_veri = {}
        for col, w in widget_listesi.items():
            yeni_veri[col] = w.value

        yeni_veri['malinDegeriTL'] = w_malinDegeri.value
        yeni_veri['failYasi'] = w_failYasi.value
        yeni_veri['sanikTutumu'] = s_cat
        yeni_veri['avukatStratejisi'] = a_cat

        # --- ENCODING (Sayıya Çevirme) ---
        df_input = pd.DataFrame([yeni_veri])
        df_input = df_input[X_ref_raw.columns] # Sıralama garantisi

        df_combined = pd.concat([X_ref_raw, df_input], axis=0, ignore_index=True)

        for col in df_combined.columns:
            if df_combined[col].dtype == 'object':
                le = LabelEncoder()
                df_combined[col] = le.fit_transform(df_combined[col].astype(str))

        son_vektor = df_combined.iloc[[-1]]

        # --- TAHMİN VE ÇEVİRİ (DECODING) ---
        try:
            # 1. Sayısal Tahmin (Örn: 2)
            tahmin_sayisal = model_orman.predict(son_vektor)[0]

            # [YENİ EKLENEN KISIM] 2. Metne Çevirme (Örn: 2 -> "Hapis")
            tahmin_metin = le_hedef.inverse_transform([tahmin_sayisal])[0]

            print(" ")
            print("="*60)
            print(f"YAPAY ZEKA KARARI:  {tahmin_metin.upper()} ")
            print("="*60)

        except Exception as e:
            print(f"Tahmin Hatası: {e}")

In [ ]:
btn_tahmin.on_click(tahmin_et)

In [ ]:
ui_rows = []
items = list(widget_listesi.values()) + [w_malinDegeri, w_failYasi]
for i in range(0, len(items), 2):
    ui_rows.append(widgets.HBox(items[i:i+2]))

ui = widgets.VBox(
    [widgets.HTML("<h3>🚀 Suç Analiz ve Tahmin Sistemi</h3>")] +
    ui_rows +
    [widgets.Label("= İfadeler -"), w_sanikIfadesi, w_avukatSavunmasi, btn_tahmin, out_panel]
)

In [ ]:
display(ui)